In [9]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import time
from datetime import datetime
from sklearn.metrics import pairwise

In [10]:
clicked = False
addspace = False
start = 0
end = 0
ustart = 0
record = ""
mtot = {
        "" : "",
        ".-" : "A",
        "-..." : "B",
        "-.-." : "C",
        "-.." : "D",
        "." : "E",
        "..-." : "F",
        "--." : "G",
        "...." : "H",
        ".." : "I",
        ".---" : "J",
        "-.-" : "K",
        ".-.." : "L",
        "--" : "M",
        "-." : "N",
        "---" : "O",
        ".--." : "P",
        "--.-" : "Q",
        ".-." : "R",
        "..." : "S",
        "-" : "T",
        "..-" : "U",
        "...-" : "V",
        ".--" : "W",
        "-..-" : "X",
        "-.--" : "Y",
        "--.." : "Z",
    }

In [11]:
img = np.ones((600, 600, 3), dtype=np.uint8)*255

In [12]:
def add(element):
    global record
    record += element

In [13]:
def identify(time):
    global record
    if time > 0.35 and time < 1.5:
        record += "-"
    else:
        record += "."

In [14]:
def morse(event, x, y, flags, param):
    global clicked, start, end, ustart
    
    if event == cv2.EVENT_LBUTTONDOWN:
        print("down")
        clicked = False
        start = time.time()
        
        
    elif event == cv2.EVENT_LBUTTONUP:
        clicked = True
        print("up")
        end = time.time()
        identify(end - start)
        ustart = time.time()

In [15]:
def morsetotext(letters):
    text = ""
    for letter in letters:
        text += mtot[letter]
    return text
    

In [2]:
background = None
avg_closed_radius = 0
accumulated_weight = 0.5

roi_top = 30
roi_bottom = 400
roi_left = 300
roi_right = 600

In [3]:
def calc_accum_avg(frame, accumulated_weight):
    global background
    
    if background is None:
        background = frame.copy().astype("float")
        return None
    
    cv2.accumulateWeighted(frame, background, accumulated_weight)

In [4]:
def segment(frame, threshold = 10):
    global background
    
    difference = cv2.absdiff(background.astype("uint8"), frame)
    
    ret, thresholded_img = cv2.threshold(difference, threshold, 255, cv2.THRESH_BINARY)
    
    image, contours, hierarchy = cv2.findContours(thresholded_img.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours) == 0:
        return None
    
    else:
        
        hand_segment = max(contours, key=cv2.contourArea)
        
        return (thresholded_img, hand_segment)

In [5]:
def intersection(A, B, C, D):
    a1 = B[1] - A[1]
    b1 = A[0] - B[0]
    c1 = a1*(A[0]) + b1*(A[0])
    
    a2 = D[1] - C[1]
    b2 = C[0] - D[0]
    c2 = a2*(C[0]) + b2*(C[1])
    
    det = a1*b2 - a2*b1
    
    x = int((b2*c1 - b1*c2)/det)
    y = int((a1*c2 - a2*c1)/det)
    
    return tuple([x, y])

In [6]:
def open_or_close(thresholded, hand_segment):
    global avg_closed_radius
    
    convex_hull = cv2.convexHull(hand_segment)   
    top  = tuple(convex_hull[convex_hull[:,:,1].argmin()][0])
    left  = tuple(convex_hull[convex_hull[:,:,0].argmin()][0])
    right = tuple(convex_hull[convex_hull[:,:,0].argmax()][0])
    
    max_y = np.max(convex_hull[:, : , 1])
    bottom_points = convex_hull[convex_hull[:, :, 1] == max_y]
    bottomleft = bottom_points[bottom_points[:, 0].argmin()]
    bottomright = bottom_points[bottom_points[:, 0].argmax()]
    bottom = [(bottomleft[0] + bottomright[0])//2, bottomleft[1]]
    bottom = tuple(bottom)
    
#     cX = (left[0] + right[0]) // 2
#     cY = (top[1] + bottom[1]) // 2

    center = intersection(top, bottom, left, right)
    
    distance = pairwise.euclidean_distances([center], Y = [left, right, top])[0]
    
    max_distance = distance.max()
    
    radius = int(max_distance * 0.80)
    
    if radius > avg_closed_radius + 5:
        openorclose = True
    else:
        openorclose = False
    
    return top, bottom, left, right, center, radius, openorclose

In [16]:
cam = cv2.VideoCapture(cv2.CAP_DSHOW)

num_frames = 0
start_calib = False
calibrated = False
opened = False
calib_frames = 0

if cam.isOpened():
    while(cam.isOpened()):
        ret, frame = cam.read()
        if ret:
            k = cv2.waitKey(1)
    
            frame = cv2.flip(frame, 1)

            frame_copy = frame.copy()

            frame_copy1 = frame.copy()

            frame_copy2 = frame.copy()

            roi = frame[roi_top:roi_bottom, roi_left:roi_right]

            gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
            gray = cv2.GaussianBlur(gray, (7,7), 0)

            if num_frames<60:
                calc_accum_avg(gray, accumulated_weight)
                if num_frames <= 59:
                    cv2.putText(frame_copy2, "Calibrating Background", (100, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                    cv2.imshow("Original Image", frame_copy2)
            else:
                hand = segment(gray,  13)

                if hand is not None:

                    thresholded, hand_segment = hand

                    cv2.drawContours(frame_copy2, [hand_segment + (roi_left, roi_top)], -1, (0, 0, 255), 1)
                    cv2.imshow("Thresholded Image", thresholded)
                    cv2.imshow("Original Image", frame_copy2)
                    
                    top, bottom, left, right, center, radius, openorclose = open_or_close(thresholded, hand_segment)
                    
                    cv2.line(frame_copy2, top, bottom, (0, 0, 255), 2)
                    cv2.line(frame_copy2, left, right, (0, 0, 255), 2)
                    cv2.circle(frame_copy2, (top[0] + roi_left, top[1] + roi_top), 3, (0, 255, 0), -1)
                    cv2.circle(frame_copy2, (bottom[0] + roi_left, bottom[1] + roi_top), 3, (0, 255, 0), -1)
                    cv2.circle(frame_copy2, (left[0] + roi_left, left[1] + roi_top), 3, (0, 255, 0), -1)
                    cv2.circle(frame_copy2, (right[0] + roi_left, right[1] + roi_top), 3, (0, 255, 0), -1)
                    cv2.circle(frame_copy2, (center[0] + roi_left, center[1] + roi_top), 3, (255, 0, 0), -1)
                    cv2.circle(frame_copy2, (center[0] + roi_left, center[1] + roi_top), radius, (255, 0, 0), 3)

            if k == ord('s'):
                start_calib = True
                print("Started Calibrating")

            if start_calib and calib_frames < 60:
                avg_closed_radius += radius
                cv2.putText(frame_copy2, "Calibrating Closed Hand Radius", (10, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            elif start_calib and calib_frames == 60:
                avg_closed_radius /= 60
                start_calib = False
                print(avg_closed_radius)
                calibrated = True

            if calibrated:
                if openorclose:
                    cv2.putText(frame_copy2, "OPEN", (10, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                    opened = True
                else:
                    cv2.putText(frame_copy2, "CLOSED", (10, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                    opened = False

            cv2.rectangle(frame_copy2, (roi_left, roi_top), (roi_right, roi_bottom), (0, 0, 255), 3)
            cv2.imshow("Original Image", frame_copy2)
            cv2.imshow("Grayscaled Blurred", gray)
            num_frames += 1

            if(start_calib):
                calib_frames += 1

            cv2.imshow("Testing Mouse Callback", frame)
            cv2.setMouseCallback('Testing Mouse Callback', morse)
            if clicked and time.time() - ustart > 1.5:
                record += "    "
                ustart = time.time()
                print("Next Letter")
                clicked = False
                addspace= True

            if addspace and time.time() - ustart > 3.5:
                record.rstrip("    ")
                record += ("        ")
                ustart = time.time()
                print("Added space")
                clicked = False
                addspace = False

            cv2.putText(img,text=record,org=(10,500), fontFace=cv2.FONT_HERSHEY_SIMPLEX,fontScale= 1,color=(255,0,0),thickness=2,lineType=cv2.LINE_AA)
            cv2.imshow('Testing Mouse Callback',frame)
            cv2.imshow('Output', img)

            k = cv2.waitKey(1)
            if k == 27:
                break

            # Clear all colors and start over if 'c' is pressed
            elif k == ord('c'):
                record = ""
                clicked = False
                img = np.ones((600, 600, 3), dtype=np.uint8)*255
        else:
            break
else:
    print("Error Opening Cam")

cam.release()
cv2.destroyAllWindows()

down
up
down
up
down
up
Next Letter
down
up
down
up
down
up
down
up
down
up
down
up
Next Letter
Added space
down
up
down
up
down
up
down
up
down
up
Next Letter
down
up
down
up
down
up
Next Letter
down
up
down
up
down
up
Next Letter
down
up
Next Letter
Added space
down
up
down
up
down
up
Next Letter
down
up
down
up
Next Letter
down
up
Next Letter


In [17]:
words = record.split("        ")
sentence = ""
for word in words:
    letters = word.split("    ")
    sentence += morsetotext(letters)
    sentence += " "
print(sentence)

SOSE SAT 
